In [1]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import tensorflow.lite as tflite
from tensorflow import keras
import zlib
from platform import python_version
import tensorflow_model_optimization as tfmot   
import tempfile
print(f"Python version used to excute the code is {python_version()}")



Python version used to excute the code is 3.7.11


In [2]:
# ######################################################## Input Parameters #########################################################
# parser = argparse.ArgumentParser()
# parser.add_argument('--model', type=str, required=True, help='model name')
# parser.add_argument('--mfcc', action='store_true', help='use MFCCs')
# args = parser.parse_args()

######################################################## Inputs and model selection  #########################################################

In [12]:
version = "a"
m = "ds_cnn"   # model name [ mlp , cnn , ds_cnn  ]
mfcc = True    # True --> excute mfcc , False --> excute STFT
alpha = 1
sparsity = 0.9
Structured = False

if Structured == True :
    model_version = f"_V_{version}_alpha={alpha}"
else :
    alpha = 1
    model_version = f"_V_{version}_Sparcity ={sparsity}"
mymodel = m + model_version
TFLITE = mymodel + ".tflite"                                    # path for saving the best model after converted to TF.lite model 
units = 8                                                       # The number of output class [8:without silence , 9 : with silence]

In [38]:

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)


# data_dir = os.path.join('.', 'data', 'mini_speech_commands_silence')
# # else:
zip_path = tf.keras.utils.get_file(
    origin="http://storage.googleapis.com/download.tensorflow.org/data/mini_speech_commands.zip",
    fname='mini_speech_commands.zip',
    extract=True,
    cache_dir='.', cache_subdir='data')

data_dir = os.path.join('.', 'data', 'mini_speech_commands')

# filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
# filenames = tf.random.shuffle(filenames)
# num_samples = len(filenames)
train_files = []
with open('readme.txt') as f:
    lines = f.readlines()
    train_files.append(lines)
######################################################## Reading File names   #########################################################
# train_files =  tf.io.gfile.glob(str(data_dir) + '/*/*')( , dtype= str)                                             
# val_files = np.loadtxt("kws_val_split.txt" , dtype= str)  
# test_files = np.loadtxt("kws_test_split.txt" , dtype= str)  
# print(f"train_files shape is {train_files.shape}")
# print(f"test_files shape is {test_files.shape}")
# print(f"val_files shape is {val_files.shape}")

# total = 8000          # NUMBER OF TOTAL Files Including THE Silence records 

# train_files =  filenames[0: int(total*0.8)]                                            # filenames[:int(total*0.8)]
# val_files = filenames[int(total*0.8): int(total*0.9)]
# test_files = filenames[int(total*0.9):]

# with silence ['stop', 'up', 'yes', 'right', 'left', 'no', 'silence', 'down', 'go']
LABELS = np.array(['stop', 'up', 'yes', 'right', 'left', 'no',  'down', 'go'] , dtype = str) 
print (LABELS)
# LABELS = LABELS[LABELS != 'README.md']

train_files shape is (6400,)
test_files shape is (800,)
val_files shape is (800,)
['stop' 'up' 'yes' 'right' 'left' 'no' 'down' 'go']


In [39]:
type(train_files[0])

numpy.str_

In [37]:
x = tf.strings.split(train_files[0] , "/")
for path in x :
    print("&" * 100 , "\n" , path, type(path))

&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& 
 tf.Tensor(b'.', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& 
 tf.Tensor(b'data', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& 
 tf.Tensor(b'mini_speech_commands', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& 
 tf.Tensor(b'down', shape=(), dtype=string) <class 'tensorflow.python.framework.ops.EagerTensor'>
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& 
 tf.Tensor(b'baeac2ba_nohash_0.wav', shape=(), dtype=string) <class 'tensorflow.pytho

######################################################## Create the SignalGenerator #########################################################

In [10]:

class SignalGenerator:
    def __init__(self, labels, sampling_rate, frame_length, frame_step,
            num_mel_bins=None, lower_frequency=None, upper_frequency=None,
            num_coefficients=None, mfcc=False):
        self.labels = labels
        self.sampling_rate = sampling_rate                                             # 16000  
        self.frame_length = frame_length                                               # 640 
        self.frame_step = frame_step                                                   # 320 
        self.num_mel_bins = num_mel_bins                                               # 40 
        self.lower_frequency = lower_frequency                                         # 20 
        self.upper_frequency = upper_frequency                                         # 4000
        self.num_coefficients = num_coefficients                                       # 10 
        num_spectrogram_bins = (frame_length) // 2 + 1                                  # ( frame size // 2 ) + 1 

        '''
        STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
        MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}
        '''

        if mfcc is True:                                                                # Remember we need to compute this matrix once so it will be a class argument 
            self.linear_to_mel_weight_matrix = tf.signal.linear_to_mel_weight_matrix(
                    self.num_mel_bins, num_spectrogram_bins, self.sampling_rate,
                    self.lower_frequency, self.upper_frequency)
            self.preprocess = self.preprocess_with_mfcc
        else:
            self.preprocess = self.preprocess_with_stft

    def read(self, file_path):
        parts = tf.strings.split(file_path, os.path.sep)
        label = parts[-2]                                  # -1 is audio.wav so 
        label_id = tf.argmax(label == self.labels)
        audio_binary = tf.io.read_file(file_path)
        audio, _ = tf.audio.decode_wav(audio_binary)
        audio = tf.squeeze(audio, axis=1)

        return audio, label_id

    def pad(self, audio):
        # Padding for files with less than 16000 samples
        zero_padding = tf.zeros([self.sampling_rate] - tf.shape(audio), dtype=tf.float32)     # if the shape of the audio is already = 16000 (sampling rate) we will add nothing 

        # Concatenate audio with padding so that all audio clips will be of the  same length
        audio = tf.concat([audio, zero_padding], 0)
        # Unify the shape to the sampling frequency (16000 , )
        audio.set_shape([self.sampling_rate])

        return audio

    def get_spectrogram(self, audio):
        stft = tf.signal.stft(audio, frame_length=self.frame_length,
                frame_step=self.frame_step, fft_length=self.frame_length)
        spectrogram = tf.abs(stft)

        return spectrogram

    def get_mfccs(self, spectrogram):
        mel_spectrogram = tf.tensordot(spectrogram,
                self.linear_to_mel_weight_matrix, 1)
        log_mel_spectrogram = tf.math.log(mel_spectrogram + 1.e-6)
        mfccs = tf.signal.mfccs_from_log_mel_spectrograms(log_mel_spectrogram)
        mfccs = mfccs[..., :self.num_coefficients]

        return mfccs

    def preprocess_with_stft(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        spectrogram = tf.expand_dims(spectrogram, -1)
 
   # expand_dims will not add or reduce elements in a tensor, it just changes the shape by adding 1 to dimensions. For example, a vector with 10 elements could be treated as a 10x1 matrix.
    #The situation I have met to use expand_dims is when I tried to build a ConvNet to classify grayscale images. The grayscale images will be loaded as matrix of size [320, 320]. However,
    #tf.nn.conv2d require input to be [batch, in_height, in_width, in_channels], 
    # where the in_channels dimension is missing in my data which in this case should be 1. So I used expand_dims to add one more dimension.
    
        spectrogram = tf.image.resize(spectrogram, [32, 32])

        return spectrogram, label

    def preprocess_with_mfcc(self, file_path):
        audio, label = self.read(file_path)
        audio = self.pad(audio)
        spectrogram = self.get_spectrogram(audio)
        mfccs = self.get_mfccs(spectrogram)
        mfccs = tf.expand_dims(mfccs, -1)

        return mfccs, label

    def make_dataset(self, files, train):
        ds = tf.data.Dataset.from_tensor_slices(files)
        ds = ds.map(self.preprocess, num_parallel_calls = tf.data.experimental.AUTOTUNE) # better than 4 tf.data.experimental.AUTOTUNE
        ds = ds.batch(32)
        ds = ds.cache()
        ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True)

        return ds


######################################################## Options for MFCC & STFT #########################################################

In [13]:
STFT_OPTIONS = {'frame_length': 256, 'frame_step': 128, 'mfcc': False}
MFCC_OPTIONS = {'frame_length': 640, 'frame_step': 320, 'mfcc': True,
        'lower_frequency': 20, 'upper_frequency': 4000, 'num_mel_bins': 40,
        'num_coefficients': 10}
if mfcc is True:
    options = MFCC_OPTIONS
    strides = [2, 1]
else:
    options = STFT_OPTIONS
    strides = [2, 2]



######################################################## Generate Data set splits #########################################################

In [ ]:
## We need the silence file to complete this 

In [14]:
generator = SignalGenerator(LABELS, 16000, **options)
train_ds = generator.make_dataset(train_files, True)
val_ds = generator.make_dataset(val_files, False)
test_ds = generator.make_dataset(test_files, False)

In [25]:
train_ds

<ShuffleDataset shapes: ((None, 49, 10, 1), (None,)), types: (tf.float32, tf.int64)>

In [24]:
for iteration, item in enumerate(it):
    print(iteration)

InvalidArgumentError: slice index -1 of dimension 0 out of bounds.
	 [[{{node strided_slice}}]]

In [19]:
# inp , label = next(it)
# print(inp)
# print(label)

InvalidArgumentError: slice index -1 of dimension 0 out of bounds.
	 [[{{node strided_slice}}]]

########################################################  building the models ########################################################

In [ ]:


mlp = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units = int(256 *alpha), activation='relu' , name =  "Dense-1" ),
    tf.keras.layers.Dense(units = int(256 *alpha), activation='relu', name =  "Dense-2"),
    tf.keras.layers.Dense(units = int(256 *alpha), activation='relu', name =   "Dense-3" ),
    tf.keras.layers.Dense(units = units , name =  "Output-Layer")                                   # change to 9 if silence included 
])

cnn = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=int(128 *alpha), kernel_size=[3,3], strides=strides, use_bias=False , name = "Conv2D-1"),
    tf.keras.layers.BatchNormalization(momentum=0.1 , name = "Btch_Norm-1"),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 *alpha), kernel_size=[3,3], strides=[1,1], use_bias=False , name = "Conv2D-2"),
    tf.keras.layers.BatchNormalization(momentum=0.1 , name = "Btch_Norm-2"),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 *alpha), kernel_size=[3,3], strides=[1,1], use_bias=False , name = "Conv2D-3"),
    tf.keras.layers.BatchNormalization(momentum=0.1 , name = "Btch_Norm-3"),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D( name =  "GlobalAveragePooling-Layer"),
    tf.keras.layers.Dense(units = units, name =  "Output-Layer")
])

ds_cnn = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=int(256 *alpha), kernel_size=[3,3], strides=strides, use_bias=False, name = "Conv2D-1"),
    tf.keras.layers.BatchNormalization(momentum=0.1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False, name = "DepthwiseConv2D-1"),
    tf.keras.layers.Conv2D(filters=int(256 *alpha), kernel_size=[1,1], strides=[1,1], use_bias=False, name = "Conv2D-2"),
    tf.keras.layers.BatchNormalization(momentum=0.1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.DepthwiseConv2D(kernel_size=[3, 3], strides=[1, 1], use_bias=False, name = "DepthwiseConv2D-2"),
    tf.keras.layers.Conv2D(filters=int(256 *alpha), kernel_size=[1,1], strides=[1,1], use_bias=False, name = "Conv2D-3"),
    tf.keras.layers.BatchNormalization(momentum=0.1),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D( name =  "GlobalAveragePooling-Layer"),
    tf.keras.layers.Dense(units = units, name =  "Output-Layer")
])


MODELS = {'mlp'+ model_version : mlp, 'cnn'+ model_version: cnn, 'ds_cnn'+ model_version: ds_cnn}
print(MODELS.keys())

######################################################## Define optimizer & Losses & Metrics ########################################################

In [ ]:

model = MODELS[mymodel]              # initiate the selected model 

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.Adam()
metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]



########################################################################################################################################################################
if Structured == False :                                # if False -->  Magnitude Based Pruning is applied 
    input_shape = (32, 49, 10, 1)
    model.build(input_shape)
    # Define the sparsity scheduler
    pruning_params = {'pruning_schedule':
    tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=0.30,
    final_sparsity=sparsity,
    begin_step=len(train_ds)*5,
    end_step=len(train_ds)*15)}

    prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

    model = prune_low_magnitude(model, **pruning_params)
    
    # Define the pruning callback
    PruningCallback = [tfmot.sparsity.keras.UpdatePruningStep()]
########################################################################################################################################################################    
################### Compiling the model :

model.compile(loss = loss, optimizer = optimizer, metrics = metrics)

######################################################## check points depending on preprocessing STFT , MFCC 
if mfcc is False:
    checkpoint_filepath = f'./checkpoints/stft/chkp_best_{mymodel}'

else:
    checkpoint_filepath = f'./checkpoints/mfcc/chkp_best_{mymodel}'
    
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,           
    monitor='val_sparse_categorical_accuracy',
    verbose=1,
    mode='max',
    save_best_only=True,
    save_freq='epoch')

######################################################## Model Training ########################################################

In [ ]:
if Structured == True :
    history = model.fit(train_ds, epochs=10,   validation_data=val_ds,callbacks=[model_checkpoint_callback ])
else :
    history = model.fit(train_ds, epochs=10,   validation_data=val_ds,callbacks=[model_checkpoint_callback,PruningCallback])
    # model = tf.keras.models.load_model(filepath = checkpoint_filepath)
    model_to_export = tfmot.sparsity.keras.strip_pruning(model)



In [ ]:
############################## Print Model Summary ####################
print(model.summary())

In [ ]:
import matplotlib.pyplot as plt
def plot_loss(history):
    plt.plot(history.history['sparse_categorical_accuracy'], label='Accuracy')
    plt.plot(history.history['val_sparse_categorical_accuracy'], label='val_Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig(mymodel+".png")

plot_loss(history)

In [ ]:
_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_to_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

In [ ]:

_, model_for_pruning_accuracy = model.evaluate(test_ds , verbose=0)

# print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

# print(model_to_export)

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_to_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
      f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)